# 🎨 Fashion Accessory Recommender - Outfit Encoder Training

**Author:** Pabasara11  
**Date:** 2025-12-19  
**GPU:** Tesla T4 (Free Google Colab)  
**Training Time:** 6-8 hours  

---

## 📊 What This Notebook Does:

1. ✅ Connects to your GitHub repository
2. ✅ Downloads the fashion dataset from Google Drive
3. ✅ Trains ResNet-152 outfit feature extractor
4. ✅ Saves model checkpoints to Google Drive
5. ✅ Generates training statistics and visualizations

---

## ⚙️ Setup Requirements:

- Google Account (free)
- GitHub repository:  `fashion-accessory-recommender`
- Dataset uploaded to Google Drive

---

## 🔧 STEP 1: Environment Setup

In [ ]:
# Check GPU availability
import torch
print("🔍 Checking GPU availability...")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available:  {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    device = 'cuda'
else:
    print("⚠️ No GPU detected!  Go to Runtime > Change runtime type > T4 GPU")
    device = 'cpu'

In [ ]:
# Install required packages
print("📦 Installing dependencies...")
!pip install -q transformers timm pillow tqdm

## 💾 STEP 2: Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create project folder in Google Drive
import os
project_drive = '/content/drive/MyDrive/FashionAI'
os.makedirs(project_drive, exist_ok=True)
os.makedirs(f"{project_drive}/checkpoints", exist_ok=True)
os.makedirs(f"{project_drive}/data", exist_ok=True)
print(f"✅ Google Drive mounted at: {project_drive}")

## 📥 STEP 3: Clone GitHub Repository

In [ ]:
# Clone your repository
!git clone https://github.com/Pabasara11/fashion-accessory-recommender.git
%cd fashion-accessory-recommender

# Verify structure
!ls -la backend/models/
! ls -la backend/core/

## 📊 STEP 4: Upload & Extract Dataset

**OPTION A:** Upload from your local machine (if dataset < 2GB)  
**OPTION B:** Download directly from Kaggle (recommended)

In [ ]:
# OPTION B: Download from Kaggle (RECOMMENDED)

# 1. Get Kaggle API token:
#    - Go to https://www.kaggle.com/settings
#    - Click "Create New API Token"
#    - Download kaggle.json

# 2. Upload kaggle.json to Colab (use file upload button on left sidebar)

# 3. Set up Kaggle credentials
!mkdir -p ~/.kaggle
!cp kaggle. json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 4. Download dataset
print("📥 Downloading dataset from Kaggle...")
!kaggle datasets download -d paramaggarwal/fashion-product-images-dataset

# 5. Extract
print("📦 Extracting dataset...")
! unzip -q fashion-product-images-dataset.zip -d data/raw/fashion-dataset
print("✅ Dataset ready! ")

In [ ]:
# Verify dataset
import os
data_path = 'data/raw/fashion-dataset'
if os.path.exists(f"{data_path}/images"):
    num_images = len(os.listdir(f"{data_path}/images"))
    print(f"✅ Found {num_images: ,} images")
if os.path.exists(f"{data_path}/styles. csv"):
    print(f"✅ Found styles.csv")
else:
    print("❌ Dataset structure incorrect!")

## 🔄 STEP 5:  Preprocess Dataset

In [ ]:
# Run preprocessing script
%cd /content/fashion-accessory-recommender/backend

# Update paths in preprocessor for Colab
import sys
sys. path.append('/content/fashion-accessory-recommender/backend')

from utils.data_preprocessor import FashionDataPreprocessor

# Initialize
preprocessor = FashionDataPreprocessor(
    raw_data_path='/content/fashion-accessory-recommender/data/raw/fashion-dataset',
    processed_data_path='/content/fashion-accessory-recommender/data/processed/accessories'
)

# Run full pipeline
stats = preprocessor.run_full_pipeline()

print("\n✅ Preprocessing complete!")
print(f"Train images: {stats['train_images']}")
print(f"Val images: {stats['val_images']}")

## 🚀 STEP 6: Train Outfit Encoder

In [ ]:
# Import training module
import sys
sys.path.append('/content/fashion-accessory-recommender/backend')

from core.train_outfit_encoder import OutfitEncoderTrainer

# Initialize trainer
trainer = OutfitEncoderTrainer(
    data_dir='/content/fashion-accessory-recommender/data/processed/accessories',
    output_dir=f"{project_drive}/checkpoints",
    device='cuda' if torch.cuda. is_available() else 'cpu'
)

# Start training
print("🚀 Starting training...")
print("⏱️ Estimated time: 6-8 hours on GPU")
print("💡 You can close this tab - training will continue! ")
print("\n" + "="*60)

trainer.train(
    num_epochs=20,
    phase1_epochs=5
)

## 📊 STEP 7: Visualize Training Results

In [ ]:
import json
import matplotlib.pyplot as plt

# Load training history
with open(f"{project_drive}/checkpoints/training_history.json", 'r') as f:
    history = json.load(f)

# Plot training curves
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Loss plot
axes[0].plot(history['train_losses'], label='Train Loss', linewidth=2)
axes[0].plot(history['val_losses'], label='Val Loss', linewidth=2)
axes[0]. set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training & Validation Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Accuracy plot
axes[1].plot(history['train_accs'], label='Train Acc', linewidth=2)
axes[1].plot(history['val_accs'], label='Val Acc', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy (%)')
axes[1].set_title('Training & Validation Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f"{project_drive}/checkpoints/training_curves.png", dpi=300, bbox_inches='tight')
plt.show()

print(f"\n🎉 Training Complete! ")
print(f"Best Validation Accuracy: {history['best_val_acc']:.2f}%")
print(f"\n📁 Saved files:")
print(f"   - best_model.pth")
print(f"   - final_model.pth")
print(f"   - training_history.json")
print(f"   - training_curves. png")

## 🧪 STEP 8: Test Trained Model

In [ ]:
from models.outfit_encoder import OutfitFeatureExtractor
from PIL import Image
import torch

# Load trained model
checkpoint = torch.load(f"{project_drive}/checkpoints/best_model.pth")

model = OutfitFeatureExtractor(
    num_classes=checkpoint['num_classes'],
    pretrained=False
)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()
model.to('cuda' if torch.cuda. is_available() else 'cpu')

print(f"✅ Model loaded!  Validation accuracy: {checkpoint['val_acc']:.2f}%")

# Test on sample image
import random
import os

val_dir = '/content/fashion-accessory-recommender/data/processed/accessories/val'
categories = os.listdir(val_dir)
sample_category = random.choice(categories)
sample_images = os.listdir(f"{val_dir}/{sample_category}")
sample_image_path = f"{val_dir}/{sample_category}/{random.choice(sample_images)}"

# Load and transform image
transform = model.get_transform()
image = Image.open(sample_image_path).convert('RGB')
image_tensor = transform(image).unsqueeze(0).to(model.backbone.fc. weight.device)

# Get prediction
with torch.no_grad():
    features, logits = model(image_tensor)
    probs = torch.softmax(logits, dim=1)
    pred_class = torch.argmax(probs, dim=1).item()
    confidence = probs[0, pred_class].item() * 100

# Display result
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
plt.imshow(image)
plt.axis('off')
plt.title(f"Predicted: {checkpoint['category_mapping'][pred_class]}\nConfidence: {confidence:.1f}%\nTrue: {sample_category}", fontsize=12)
plt.tight_layout()
plt.show()

print(f"\n✅ Model is working correctly! ")

## 💾 STEP 9: Download Trained Model

Your models are saved in Google Drive at: `/content/drive/MyDrive/FashionAI/checkpoints/`

To download to your local machine:
1. Go to Google Drive
2. Navigate to `FashionAI/checkpoints/`
3. Download `best_model.pth`
4. Place it in your local project:  `backend/models/checkpoints/`